层和块


回顾多层感知机

In [1]:
import torch
from torch import nn
from torch.nn import functional as F

In [2]:
net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256,10))

In [3]:
X = torch.rand(2, 20)

In [4]:
net(X)

tensor([[ 0.0619, -0.1388, -0.0686,  0.2351, -0.0897,  0.2578,  0.0311,  0.1986,
         -0.0372,  0.0620],
        [-0.0264, -0.1562, -0.0033,  0.1439, -0.1902,  0.2907,  0.1291,  0.1637,
         -0.1481,  0.1128]], grad_fn=<AddmmBackward0>)

自定义块

In [5]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.out = nn.Linear(256,10)
    def forward(self, X):
        return self.out(F.relu(self.hidden(X)))

当然可以。下面是对你给出的代码的详细解释：

首先，你定义了一个名为MLP的类，它继承自nn.Module。在PyTorch中，nn.Module是所有神经网络模块的基类，包括层、模型等。通过继承这个基类，你可以构建自己的神经网络模型。

__init__ 方法
__init__ 是一个特殊的方法，它在创建类的新实例时自动调用。在这个方法中，你初始化了网络的两层：

self.hidden = nn.Linear(20, 256): 这一行定义了一个全连接层（或称为线性层），它接受一个大小为20的输入向量并输出一个大小为256的向量。这个层有 20 * 256 = 5120 个权重参数和 256 个偏置参数。

self.out = nn.Linear(256, 10): 这一行定义了另一个全连接层，它接受一个大小为256的输入向量（来自self.hidden层的输出）并输出一个大小为10的向量。这个层有 256 * 10 = 2560 个权重参数和 10 个偏置参数。

super().__init__() 是对父类 nn.Module 的初始化方法的调用，这是PyTorch要求的做法，以确保父类中的所有初始化操作都得到正确的执行。

forward 方法
forward 方法定义了数据通过网络的前向传播路径。当你将一个输入传递给一个神经网络模型时，这个方法会被调用。

在这个例子中，forward 方法接受一个名为 X 的输入，该输入预期是一个大小为 (batch_size, 20) 的张量，其中 batch_size 是批次中的样本数量。

self.hidden(X): 这一行将输入 X 传递给 self.hidden 层，该层会对输入进行线性变换。

F.relu(...): 这是一个ReLU激活函数，它会对上一步的输出应用非线性变换。ReLU函数的定义是 f(x) = max(0, x)，它会将所有负值置为0，而正值则保持不变。

self.out(...): 最后，经过ReLU激活函数处理后的输出被传递给 self.out 层进行第二次线性变换，得到最终的输出。

这个 forward 方法最终返回的是一个大小为 (batch_size, 10) 的张量，其中每一行代表一个输入样本经过网络处理后的10个类别的预测得分。

总结来说，这个 MLP 类定义了一个简单的多层感知机（MLP）模型，它有一个隐藏层和一个输出层。这个模型可以接受一个大小为20的输入向量，并输出一个大小为10的向量，通常用于10类分类问题。

实例化多层感知机的层，然后在每次调用正向传播函数时调用这些层

In [6]:
net = MLP()
net(X)

tensor([[ 0.0292, -0.0904,  0.0961,  0.0554,  0.1272, -0.1335, -0.2312, -0.0484,
         -0.2578,  0.0414],
        [-0.0105, -0.0194,  0.1360, -0.0881, -0.0278, -0.1685, -0.3030, -0.0401,
         -0.1090,  0.0154]], grad_fn=<AddmmBackward0>)

顺序块

In [7]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for block in args:
            self._modules[block] = block
    
    def forward(self, X):
        for block in self._modules.values():
            X = block(X)
        return X


In [8]:
net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256,10))

net(X)

tensor([[ 0.1263, -0.2223, -0.0129,  0.1760,  0.0761,  0.1645,  0.1349, -0.2172,
         -0.0012,  0.0541],
        [ 0.1140, -0.3457, -0.0431,  0.2171,  0.0350,  0.1335,  0.1719, -0.0383,
         -0.0190, -0.0449]], grad_fn=<AddmmBackward0>)

这段代码定义了一个自定义的序贯模型MySequential，它模仿了PyTorch内置的nn.Sequential的行为。下面是对代码的详细解释：

定义MySequential类
MySequential类继承自nn.Module，这意味着它是一个可以被用在神经网络中的组件。

__init__方法
在__init__方法中，*args允许传入任意数量的参数，这些参数预期是神经网络层或其他模块。对于传入的每一个block（层或模块），它都被添加到self._modules字典中。self._modules是nn.Module的一个特殊属性，用于存储子模块。

python
def __init__(self, *args):  
    super().__init__()  
    for block in args:  
        self._modules[block] = block
这里有一点需要注意的是，通常我们会在self._modules中使用唯一的字符串键来存储子模块，但在这个例子中直接使用了模块对象作为键。虽然这在技术上是可行的，但通常不推荐这样做，因为它可能会导致混淆或不可预见的行为。

forward方法
forward方法定义了数据通过模型的前向传播路径。它遍历self._modules中的所有值（即传入的层或模块），并将输入X依次传递给这些层或模块。

python
def forward(self, X):  
    for block in self._modules.values():  
        X = block(X)  
    return X
这里，X = block(X)表示将X作为输入传递给当前的block（层或模块），并将输出重新赋值给X。这样，在遍历完所有层或模块后，X将包含最终的输出。

使用MySequential类
接下来，代码创建了一个MySequential的实例net，并传入了三个层：一个线性层、一个ReLU激活函数层和一个输出层。

python
net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
这相当于创建了一个序贯模型，其结构如下：

输入层 -> 线性层 (20 -> 256) -> ReLU激活函数 -> 线性层 (256 -> 10) -> 输出层
最后，通过调用net(X)，输入数据X将通过这个序贯模型进行前向传播，并返回最终的输出。

注意点
模块键的问题：如前所述，直接在self._modules中使用模块对象作为键可能不是一个好主意。更常见的做法是使用字符串键，如self._modules['linear'] = nn.Linear(20, 256)。

输入数据X的维度：在这段代码中，没有提及输入数据X的具体维度。但从上下文来看，我们可以假设X的形状是(batch_size, 20)，其中batch_size是批次中的样本数量。

缺少导入语句：为了使这段代码正常工作，你需要确保已经导入了必要的库和模块，例如：

python
import torch  
import torch.nn as nn
总的来说，这段代码定义了一个简单的自定义序贯模型，并展示了如何使用它来进行前向传播。

在正向传播函数中执行代码

In [9]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)
    
    def forward(self, X):
        X = self.linear(X)
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        X = self.linear(X)
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()


In [10]:
net = FixedHiddenMLP()
net(X)

tensor(-0.0393, grad_fn=<SumBackward0>)

这段代码定义了一个名为FixedHiddenMLP的神经网络模型，该模型继承自nn.Module，即PyTorch中的基础神经网络模块。下面我将详细解释这段代码的每个部分：

1. __init__ 方法
在__init__方法中，模型进行了初始化。

super().__init__(): 调用父类nn.Module的初始化方法。
self.rand_weight = torch.rand((20, 20), requires_grad=False): 创建一个20x20的随机权重矩阵，并且设置requires_grad=False，这意味着在训练过程中不会更新这个权重矩阵。
self.linear = nn.Linear(20, 20): 创建一个线性层，输入和输出特征数都是20。
注意：在self.rand_weight的初始化中，括号不匹配，应该是self.rand_weight = torch.rand((20, 20), requires_grad=False)。

2. forward 方法
forward方法定义了数据通过模型的前向传播路径。

X = self.linear(X): 输入数据X首先通过一个线性层。
X = F.relu(torch.mm(X, self.rand_weight) + 1): 然后，X与固定的随机权重矩阵self.rand_weight进行矩阵乘法，并加上1。接着，通过ReLU激活函数。
X = self.linear(X): 再次通过线性层。
while X.abs().sum() > 1: X /= 2: 如果X的L1范数（即所有元素绝对值的和）大于1，则将X的每个元素除以2，直到L1范数小于或等于1。这是一个归一化步骤，确保X的L1范数在1以内。
return X.sum(): 最后，返回X中所有元素的和。
3. 创建模型实例和调用
net = FixedHiddenMLP(): 创建一个FixedHiddenMLP的实例。
net(X): 用X作为输入调用模型。这里X应该是一个形状为(batch_size, 20)的张量，其中batch_size是批次中的样本数量。
总结
这个FixedHiddenMLP模型是一个相对简单的神经网络，它包含两个线性层和一个固定的随机权重矩阵。模型输出是输入数据经过一系列变换后的所有元素的和，且这些变换包括线性变换、ReLU激活和L1范数归一化。由于其中一个权重矩阵是固定的，所以在训练过程中不会更新。

FixedHiddenMLP这个模型的设计具有一定的特殊性和目的。固定随机权重矩阵的使用可以带来几个潜在的好处或考虑因素：

模型简化：通过使用固定的随机权重矩阵，我们消除了学习该特定权重矩阵的需要。这可以简化模型，减少训练时需要的计算资源和时间。

特征变换：尽管权重是随机的且固定的，但它们仍然可以对输入数据进行有意义的变换。这种变换可能有助于模型捕捉数据中的某些模式或结构，尤其是当这些模式不容易通过常规线性或非线性层捕捉时。

正则化：通过引入固定的随机性，模型可能具有某种形式的正则化效果。正则化是防止模型过拟合训练数据的一种技术。通过使用固定的随机权重，模型在训练过程中不会过度依赖于这些权重，这有助于保持模型的泛化能力。

模型组合：固定随机权重可以被视为一种特征工程技术，与其他可学习的层结合使用可以产生丰富的特征表示。这种组合可能使模型能够学习更复杂的函数或模式。

随机性引入：在某些情况下，引入随机性可以提高模型的性能。固定的随机权重可以看作是一种在模型架构中显式地引入随机性的方式。

计算效率：由于权重是固定的，因此在前向传播过程中不需要进行权重更新，这可以提高计算效率，特别是在资源有限或需要高效推断的场景下。

然而，固定随机权重矩阵的使用也有其局限性。由于权重是固定的，模型可能无法适应数据中的某些变化或模式，这可能会限制模型的性能。此外，固定的随机性可能不适用于所有任务或数据集，因此需要根据具体情况来决定是否使用这种技术。

总之，FixedHiddenMLP的设计是出于对计算效率、模型正则化和特定特征变换的考虑。它展示了在神经网络设计中结合可学习和固定组件的灵活性。

混合搭配各种组合块的方法

In [13]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64),
                                 nn.ReLU(),
                                 nn.Linear(64, 32),
                                 nn.ReLU()
                                )
        self.linear = nn.Linear(32, 16)
    
    def forward(self, X):
        return self.linear(self.net(X))

In [14]:
chimera = nn.Sequential(
                        NestMLP(),
                        nn.Linear(16, 20),
                        FixedHiddenMLP()
                       )
chimera(X)

tensor(0.2197, grad_fn=<SumBackward0>)

好的，我将为您详细解释这段代码。

首先，让我们从NestMLP类开始：

NestMLP类
NestMLP是一个神经网络模型，继承自nn.Module。它包含两个主要的组件：

self.net：这是一个nn.Sequential模型，它包含两个线性层（nn.Linear）和两个ReLU激活函数。数据首先通过第一个线性层，其输入特征数为20，输出特征数为64。然后，数据经过一个ReLU激活函数。接着，数据进入第二个线性层，其输入特征数为64，输出特征数为32。最后，数据再次经过一个ReLU激活函数。

python
self.net = nn.Sequential(  
    nn.Linear(20, 64),  
    nn.ReLU(),  
    nn.Linear(64, 32),  
    nn.ReLU()  
)
self.linear：这是一个线性层，其输入特征数为32（与self.net的输出特征数相匹配），输出特征数为16。

python
self.linear = nn.Linear(32, 16)
在forward方法中，数据首先通过self.net，然后通过self.linear。这两个操作是顺序进行的，即self.net的输出作为self.linear的输入。

python
def forward(self, X):  
    return self.linear(self.net(X))
Chimera模型
Chimera是一个nn.Sequential模型，它组合了三个组件：

NestMLP()：我们之前定义的NestMLP模型。
nn.Linear(16, 20)：一个线性层，其输入特征数为16（与NestMLP的输出特征数相匹配），输出特征数为20。
FixedHiddenMLP()：这是您之前提供的另一个模型，其中包含一个固定的随机权重矩阵和两个线性层。
当您使用chimera(X)调用此模型时，输入数据X首先通过NestMLP，然后通过线性层，最后通过FixedHiddenMLP。这三个操作也是顺序进行的，即每个组件的输出作为下一个组件的输入。

简而言之，Chimera模型是一个更复杂的神经网络，由NestMLP、一个线性层和FixedHiddenMLP组成，它们按顺序连接。